# Day 1: Introduction to LLMs and Prompting


**Summary:** In this notebook, we will cover how to connect to a Large Language Model and walkthrough different prompting techniques that will be useful during this workshop.

In [2]:
# !pip install langchain-openai      # Langchain OpenAI package
# !pip install rich                  # Helps print doc string
# !pip install python-dotenv         # Helps hide envionrment API Keys

In [7]:
import os # operating system
from rich import inspect # pretty print doctring

# Connect to OpenAI models
from langchain_openai import ChatOpenAI

In [8]:
from dotenv import load_dotenv # load enviornment variables
load_dotenv()

True

In [9]:
# connect to OpenAI
openai_api_key = os.environ.get("OPENAI_API_KEY")
openai_organization = os.environ.get("OPENAI_ORGANIZATION")

## Section 1: Intro to Langchain
---

## Step 1: Connecting to OpenAI

First we need to connect to a large language model 

In [11]:
llm = ChatOpenAI(
    openai_api_key = openai_api_key, 
    openai_organization = openai_organization,
    model = "gpt-4o-mini")

In [12]:
llm.invoke("hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-12f48247-7384-4f43-ae54-e902f3cbdc34-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Step 2: Let's take a closer look at `ChatOpenAI`

The API we will be using to connect to ChatGPT is called [LangChain](https://python.langchain.com/docs/introduction/).
> LangChain is a framework for developing applications powered by large language models (LLMs).

Langchain is a easy way of connecting to ChatGPT so we can start building application on top of ChatGPT. Throughout this workshop we will predominantly use this library to build our application.

Now let's take a closer look at `ChatOpenAI`, which is how we connect to ChatGPT. In this section, I'll go through some of the most important parameters to set for this function, but here is the the documentation if you want to dive deeper into the API. 
+ Ref: https://sj-langchain.readthedocs.io/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html


### Parameters

1. **openai_api_key** and **openai_organization** - will be provided to you, which you will need to log into your account. Think of this as user name and password.
2. **model** - here we select the model that we want to use for our workshop
3. **temperature** - adjust the creativity of your response. With a lower temperature the model is more conservative with it's reponse, usually resulting in shorter more concise answers. With a high temperature models become more creative and "talk" more. Usually, resulting in higher word count for a response
4. **max_token** - maximum number of words you want to get back from the LLM.

This is enough to get started, but eventually you might want to add more configurations. See [link](https://sj-langchain.readthedocs.io/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html).

In [15]:
llm2 = ChatOpenAI(
    openai_api_key = openai_api_key, 
    openai_organization = openai_organization,
    model = "gpt-4o-mini",
    temperature=0.01)

In [16]:
response = llm2.invoke("What is the capital of Armenia?")

In [17]:
response

AIMessage(content='The capital of Armenia is Yerevan.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'finish_reason': 'stop', 'logprobs': None}, id='run-804d9bb2-5ddb-4484-89c6-e78a6bcc982c-0', usage_metadata={'input_tokens': 14, 'output_tokens': 10, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Let take a closer look at what Langchain's `ChatOpenAI` returns

`AIMessage` is the object that is returned after we make an API call to GTP-4o-mini.
We can access the message content, response metadata, the model we used to make the API call, a unique ID of the message we sent and more

In [18]:
print("Answer:     ", response.content)
print("Model name: ", response.response_metadata["model_name"])
print("ID:         ",response.id)

Answer:      The capital of Armenia is Yerevan.
Model name:  gpt-4o-mini-2024-07-18
ID:          run-804d9bb2-5ddb-4484-89c6-e78a6bcc982c-0


# Section 2: Intro to Prompting

What is prompting?
> Prompt engineering is a relatively new discipline for developing and optimizing prompts to efficiently use language models (LMs) for a wide variety of applications and research topics. Prompt engineering skills help to better understand the capabilities and limitations of large language models (LLMs). - Prompt Engineering Guide


In this section, we are going to introduce more effective way's of putting toghther questions to send to ChatGPT. I'm going to go through some techniques that you have probably already used and some techniques that will be new to you, but will improve how well ChatGPT respones to your questions. 


**Summary:** Prompting techniques we will cover in this section.
1. Zero-shot prompting
2. Few-shot prompting
3. Chain-of-Thought prompting
4. Meta-prompting

But first - let's look at how we can format prompts in Langchain

What are `PromptTemplate`?
>Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.


**References**
+ https://www.promptingguide.ai/
+ Zero-shot/Few-shot paper: [Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165)
+ Chain-of-Thought paper: [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/pdf/2201.11903)
+ Meta-prompting paper: [Meta Prompting for AI Systems](https://arxiv.org/pdf/2311.11482)

In [19]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

In [20]:
# PromptTemplate - 
prompt_template = PromptTemplate.from_template(
    "What is the capital of {country}?"
)

prompt_template.pretty_print()

What is the capital of {country}?


In [21]:
chain = prompt_template | llm2

print("---------------------------")
print(chain.invoke({"country": "Armenia"}))
print("---------------------------")
print(chain.invoke({"country": "France"}))
print("---------------------------")
print(chain.invoke({"country": "Germany"}))

---------------------------
content='The capital of Armenia is Yerevan.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None} id='run-71df1d9e-4e58-43e2-bf5c-688a34f4aa6f-0' usage_metadata={'input_tokens': 14, 'output_tokens': 10, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
---------------------------
content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_deta

**Note:** My adding a simple prompt template we can reuse our prompt to ask the same question, but in difference ways.

### Technique 1: Zero-shot Prompting

**Zero-shot prompting** is just asking your model a question. No context, no instructions, just asking a question to get the answer you want.

>Zero-shot prompting means that the prompt used to interact with the model won't contain examples or demonstrations. The zero-shot prompt directly instructs the model to perform a task without any additional examples to steer it.

In [24]:
llm2.invoke("""
Classify the text into neutral, negative or positive. 
Text: I think the vacation is okay.
Sentiment:
""").content

'Sentiment: Neutral'

### Technique 2: Few-Shot Prompting

**Few-Shot Prompting** enables your model to learn called "in-context learning" when you ask it a question.
>Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance.

In [ ]:
llm2.invoke("""
This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Positive
What a horrible show! //
""").content

### Technique 3: Chain-of-Thought Prompting


This is a prompting technique where you provide an example of how you reason through a complex problem, so that the LLM knows you solved the problem and mimics this type of reasoning.
>Introduced in Wei et al. (2022), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. 
You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

In [27]:
llm2.invoke("""
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:
""").content

"Let's identify the odd numbers in the group: 15, 5, 13, 7, and 1.\n\nNow, let's add them together:\n\n15 + 5 + 13 + 7 + 1 = 41\n\nSince 41 is an odd number, the statement is False. The odd numbers in this group do not add up to an even number."

---
## Section 2: Examples of Prompt Types

### Type 1: Zero-Shot Prompting
<div style="text-align:center;">
    <img src="../assets/zsp.png"/>
</div>

### Type 2: Few-Shot Prompting
<div style="text-align:center;">
    <img src="../assets/fsp.png"/>
</div>

### Type 3: Few-Shot Prompting
<div style="text-align:center;">
    <img src="../assets/cot.png"/>
</div>

### Type 4: Meta-Prompting
<div style="text-align:center;">
    <img src="../assets/metap.png"/>
</div>